### IRI assigner

After getting your initial DDM in the same format as the file "format_example.json", run this program to assign IRIs to each sections, paragraph, and sentence. Then, the file is ready to be processed by paper2lkg.

In [36]:
import json
import sys
from pathlib import Path

# Get the current notebook directory
CURRENT_DIR = Path().resolve()

# Add packages to sys.path
UTILITIES_RELATIVE_PATH = '../../../src'
UTILITIES_ABSOLUTE_PATH = str((CURRENT_DIR / UTILITIES_RELATIVE_PATH).resolve())
if UTILITIES_ABSOLUTE_PATH not in sys.path:
    sys.path.append(UTILITIES_ABSOLUTE_PATH)

sys.path.append('../../src/utilities')
from utilities.content_processor import sanitise_iri # type: ignore

# from content_processor import sanitise_url

In [37]:
ASKG_DATA = "https://www.anu.edu.au/data/scholarly/"
ASKG_ONTO = "https://www.anu.edu.au/onto/scholarly#"

In [38]:
dataset = "test"
index = 10

with open(f"./kg_{dataset}_{index}_org.json") as f:
    paper = json.load(f)

In [39]:
new_paper = {}


new_paper["iri"] = "Paper-" + sanitise_iri(paper["title"])
new_paper["title"] = paper["title"]
new_paper["authors"] = paper["authors"]
new_paper["keywords"] = paper["keywords"]
new_paper["sections"] = []

for i, section in enumerate(paper["sections"], 1):
    new_section = {}
    new_section["iri"] = "Section-" + str(i)
    new_section["subtitle"] = section["subtitle"]
    new_section["paragraphs"] = []
    new_paper["sections"].append(new_section)


    for j, paragraph in enumerate(section["paragraphs"], 1):
        new_paragraph = {}
        new_paragraph["iri"] = new_section["iri"] + "-Paragraph-" + str(j)
        new_paragraph["sentences"] = []
        new_section["paragraphs"].append(new_paragraph)

        for k, sentence in enumerate(paragraph["sentences"], 1):
            new_sentence = {}
            new_sentence["iri"] = new_paragraph["iri"] + "-Sentence-" + str(k)
            new_sentence["text"] = sentence
            new_paragraph["sentences"].append(new_sentence)


In [40]:
paper = new_paper

In [41]:
assert paper.keys() == {"iri", "title", "authors", "keywords", "sections"}

for section in paper["sections"]:
    assert section.keys() == {"iri", "subtitle", "paragraphs"}

    for paragraph in section["paragraphs"]:
        assert paragraph.keys() == {"iri", "sentences"}

        for sentence in paragraph["sentences"]:
            assert sentence.keys() == {"iri", "text"}

In [42]:
with open(f"./kg_{dataset}_{index}.json", "w") as f:
    json.dump(paper, f, indent=2)